In [1]:
pip install transformers==4.50

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from transformers import PegasusXConfig, PegasusXForConditionalGeneration, pipeline, AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"

2025-09-28 14:54:06.684961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759071246.876532      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759071246.930319      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [4]:
config = PegasusXConfig(max_position_embeddings = 512, activation_function="silu")

# Fine Tuning

In [5]:
from datasets import load_dataset

ds_path="/kaggle/input/cnndailymail/train.parquet"
ds = load_dataset('parquet', data_files=ds_path)


Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-base")

# Define the preprocessing function
def preprocess_function(examples):
    # Tokenize the reports (input)
    model_inputs = tokenizer(
        examples['article'],
        max_length=128,
        truncation=True,
    )

    # Tokenize the summaries (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['highlights'],
            max_length=128,
            truncation=True,
        )

    # Add the labels to the model inputs
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing function to the entire dataset
#tokenized_dataset = combined_dataset.map(preprocess_function, batched=True)
tokenized_dataset = ds.map(preprocess_function, batched=True)
train_dataset_split = tokenized_dataset['train']

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
import torch
from transformers import PegasusXForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Load the model
model = PegasusXForConditionalGeneration.from_pretrained("google/pegasus-x-base",config=config, ignore_mismatched_sizes=True).to(device)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",            # The output directory
    per_device_train_batch_size=1,     # Batch size for training
    num_train_epochs=3,                # Number of training epochs
    logging_dir='./logs',              # Directory for logs
    logging_steps=10,
    save_strategy="epoch",
    report_to="tensorboard" # Recommended for more advanced visualization
)





pytorch_model.bin:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Some weights of PegasusXForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-x-base and are newly initialized: ['model.decoder.layers.12.encoder_attn.k_proj.weight', 'model.decoder.layers.12.encoder_attn.out_proj.weight', 'model.decoder.layers.12.encoder_attn.q_proj.weight', 'model.decoder.layers.12.encoder_attn.v_proj.weight', 'model.decoder.layers.12.encoder_attn_layer_norm.bias', 'model.decoder.layers.12.encoder_attn_layer_norm.weight', 'model.decoder.layers.12.fc1.bias', 'model.decoder.layers.12.fc1.weight', 'model.decoder.layers.12.fc2.bias', 'model.decoder.layers.12.fc2.weight', 'model.decoder.layers.12.final_layer_norm.bias', 'model.decoder.layers.12.final_layer_norm.weight', 'model.decoder.layers.12.self_attn.k_proj.weight', 'model.decoder.layers.12.self_attn.out_proj.weight', 'model.decoder.layers.12.self_attn.q_proj.weight', 'model.decoder.layers.12.self_attn.v_proj.weight', 'model.decoder.layers.12.self_attn_layer_norm.bias', 'model.decod

generation_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

In [8]:
train_dataset_split = train_dataset_split.select(range(16000))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_split,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_19/1111766111.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# Start fine-tuning!
trainer.train()

Step,Training Loss
10,10.289600
20,9.600800
30,9.275600
40,8.884500
50,8.857600
60,8.413500
70,8.273100
80,8.240300
90,7.805900
100,7.857200


TrainOutput(global_step=48000, training_loss=5.699961366415024, metrics={'train_runtime': 17301.4887, 'train_samples_per_second': 2.774, 'train_steps_per_second': 2.774, 'total_flos': 1.7318696402681856e+16, 'train_loss': 5.699961366415024, 'epoch': 3.0})

In [10]:
!mkdir -p results

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
!rm -r results

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
# Assuming 'trainer' is your Hugging Face Trainer object
output_dir = "/kaggle/working/silu"
trainer.save_model(output_dir)

# Testing

In [13]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# model_ckpt = "/kaggle/working/silu/"
# tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

# pipe = pipeline('summarization', model = model_ckpt)
# pipe_out = pipe(test_text)
# print(pipe_out)